En este Notebook cargaré las bases de datos del Atlas de Harvard de Complejidad Economica (https://atlas.cid.harvard.edu/explore/) y procederé a hacer un ETL.

# Opcion 1: cudf -- descartada

In [ ]:
!nvidia-smi

Mon Jul 22 22:52:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install cudf-cu12 --extra-index-url=https://pypi.nvidia.com #codigo con pandas lo pasa a qa. todo lo haces en pandas y lo escalas a gpus

In [ ]:
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [ ]:
%load_ext cudf.pandas

In [ ]:
import os
from tqdm import tqdm

In [ ]:
directory = "/content/drive/MyDrive/Colab Notebooks/Python/Research/dataverse_files"

In [ ]:
os.chdir(directory)

In [ ]:
list_to_use2 = [i for i in os.listdir() if (len(i) == 47) and (".csv" in i)]

df vacio e itero para cargar la data

In [ ]:
df = pd.DataFrame()
for i in tqdm(list_to_use2):
  df_temp = pd.read_csv(i)
  df = pd.concat([df, df_temp])

  2%|▏         | 1/60 [00:03<03:25,  3.48s/it]

Tarda mucho, no es óptimo. Descartado

# Preparo los datos para la opción 2

## Parte 1

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

#main libraries, building blocks
import pandas as pd
import os
import polars as pl
from tqdm import tqdm

In [ ]:
directory = "/content/drive/MyDrive/Colab Notebooks/Python/Research/dataverse_files"

In [ ]:
os.chdir(directory)

In [ ]:
len("country_partner_sitcproduct4digit_year_1967.dta")

47

In [ ]:
list_to_use = [i for i in os.listdir() if (len(i) == 47) and (".dta" in i)]

In [ ]:
sum([os.path.getsize(i)/1000000 for i in list_to_use])

9657.951328000001

In [ ]:
list_to_use2 = [i for i in os.listdir() if (len(i) == 47) and (".csv" in i)]

In [ ]:
list_to_use2

In [ ]:
import re
[re.findall(r'\d+', i)[1] for i in list_to_use]

Pasamos todo a CSV

In [ ]:
for filename in tqdm(list_to_use):
    if filename.endswith(".dta"):
        filepath = os.path.join(directory, filename)
        df_name = 'data_' + filename.split('_')[-1].split('.')[0]  # Extraer el año y crear el nombre
        pl_df = pd.read_stata(filepath)
        pl_df['Year'] = int(re.findall(r'\d+', filename)[1])
        pl_df = pl_df[pl_df["sitc_product_code"] != "ZZZZ"]
        pl_df = pl_df[pl_df["location_code"].isin(["KOR", "THA", "MYS", "IDN", "ARG", "BRA"])|pl_df["partner_code"].isin(["KOR", "THA", "MYS", "IDN", "ARG", "BRA"])]
        pl_df.to_csv(filename.replace(".dta", "2.csv"), index=False)

100%|██████████| 60/60 [10:18<00:00, 10.31s/it]


In [ ]:
list_to_use3 = [i.replace(".csv","2.csv") for i in list_to_use2]

In [ ]:
list_to_use3

In [ ]:
list_to_use2

# Pandas -- exitoso!

## Arreglar problema en formato de las entidades que no me permitía mergearlas

In [ ]:
dfs = {}

# Cargar cada archivo CSV y almacenar el dataframe en el diccionario
for file in list_to_use3:
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_csv(file_path)
        dfs[file] = df
        print(f"{file} cargado con éxito.")
    except Exception as e:
        print(f"Error al cargar {file}: {e}")

# Revisar los tipos de datos de cada dataframe
for file, df in dfs.items():
    print(f"\nTipos de datos para {file}:")
    print(df.dtypes)

country_partner_sitcproduct4digit_year_19712.csv cargado con éxito.
country_partner_sitcproduct4digit_year_20152.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19822.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19982.csv cargado con éxito.
country_partner_sitcproduct4digit_year_20002.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19702.csv cargado con éxito.
country_partner_sitcproduct4digit_year_20102.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19842.csv cargado con éxito.
country_partner_sitcproduct4digit_year_20012.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19762.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19802.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19662.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19942.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19892.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19852.csv

Acá primero se encontraron los problemas. En el output no aparecen porque lo volví a correr una vez que se arregló todo.

In [ ]:
dfs = {}
dtypes = {}

# Cargar cada archivo CSV y almacenar el dataframe y sus tipos de datos en los diccionarios
for file in list_to_use3:
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_csv(file_path)
        dfs[file] = df
        dtypes[file] = df.dtypes
        print(f"{file} cargado con éxito.")
    except Exception as e:
        print(f"Error al cargar {file}: {e}")

# Comparar los tipos de datos de cada dataframe
comparison_result = {}

# Obtenemos el primer archivo como referencia
reference_file = list_to_use3[0]
reference_dtypes = dtypes[reference_file]

for file, dtype in dtypes.items():
    if file != reference_file:
        differences = {}
        for column in dtype.index:
            if column in reference_dtypes.index:
                if dtype[column] != reference_dtypes[column]:
                    differences[column] = (reference_dtypes[column], dtype[column])
            else:
                differences[column] = (None, dtype[column])
        comparison_result[file] = differences

# Mostrar diferencias
for file, differences in comparison_result.items():
    if differences:
        print(f"\nDiferencias encontradas en {file}:")
        for column, (ref_type, file_type) in differences.items():
            print(f"  Columna '{column}': Tipo de referencia = {ref_type}, Tipo del archivo = {file_type}")
    else:
        print(f"\nNo se encontraron diferencias en {file}.")

# Verificar si todas las columnas en el archivo de referencia están presentes en los otros archivos
missing_columns = {}
for file, dtype in dtypes.items():
    if file != reference_file:
        missing_cols = set(reference_dtypes.index) - set(dtype.index)
        if missing_cols:
            missing_columns[file] = missing_cols

for file, cols in missing_columns.items():
    print(f"\nColumnas faltantes en {file}: {cols}")

country_partner_sitcproduct4digit_year_19712.csv cargado con éxito.
country_partner_sitcproduct4digit_year_20152.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19822.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19982.csv cargado con éxito.
country_partner_sitcproduct4digit_year_20002.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19702.csv cargado con éxito.
country_partner_sitcproduct4digit_year_20102.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19842.csv cargado con éxito.
country_partner_sitcproduct4digit_year_20012.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19762.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19802.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19662.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19942.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19892.csv cargado con éxito.
country_partner_sitcproduct4digit_year_19852.csv

Así solucioné los problemas de data types

In [ ]:
# Lista de archivos CSV con discrepancias en tipos de datos
csv_files_with_issues = [
    'country_partner_sitcproduct4digit_year_19702.csv',
    'country_partner_sitcproduct4digit_year_19662.csv',
    'country_partner_sitcproduct4digit_year_19692.csv',
    'country_partner_sitcproduct4digit_year_19652.csv',
    'country_partner_sitcproduct4digit_year_19682.csv',
    'country_partner_sitcproduct4digit_year_19622.csv',
    'country_partner_sitcproduct4digit_year_19642.csv',
    'country_partner_sitcproduct4digit_year_19632.csv',
    'country_partner_sitcproduct4digit_year_19672.csv'
]

# Función para convertir columnas a float64
def convert_columns_to_float64(df, columns):
    for column in columns:
        if column in df.columns:
            df[column] = df[column].astype(float)
    return df

# Procesar cada archivo CSV
for file in csv_files_with_issues:
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_csv(file_path)
        df = convert_columns_to_float64(df, ['export_value', 'import_value'])
        df.to_csv(file_path, index=False)
        print(f"{file} procesado y guardado con éxito.")
    except Exception as e:
        print(f"Error al procesar {file}: {e}")

print("Todos los archivos problemáticos han sido procesados.")

country_partner_sitcproduct4digit_year_19702.csv procesado y guardado con éxito.
country_partner_sitcproduct4digit_year_19662.csv procesado y guardado con éxito.
country_partner_sitcproduct4digit_year_19692.csv procesado y guardado con éxito.
country_partner_sitcproduct4digit_year_19652.csv procesado y guardado con éxito.
country_partner_sitcproduct4digit_year_19682.csv procesado y guardado con éxito.
country_partner_sitcproduct4digit_year_19622.csv procesado y guardado con éxito.
country_partner_sitcproduct4digit_year_19642.csv procesado y guardado con éxito.
country_partner_sitcproduct4digit_year_19632.csv procesado y guardado con éxito.
country_partner_sitcproduct4digit_year_19672.csv procesado y guardado con éxito.
Todos los archivos problemáticos han sido procesados.


## Cargo dataframes:

### Todo el dataframe

Acá probé cargar todo el dataframe solo para visualizar. Como se vé, no pude exportarlo porque era muy pesado.

In [ ]:
# Lista para almacenar cada DataFrame
dataframes = []

# Cargar cada archivo CSV y agregarlo a la lista de DataFrames
for filename in list_to_use3:
    df = pd.read_csv(filename)
    dataframes.append(df)

# Combinar todos los DataFrames en uno solo
combined_df = pd.concat(dataframes, ignore_index=True)

# Exportar el DataFrame combinado a un archivo Excel
combined_df.to_excel('combined_data.xlsx', index=False)

#print("Todos los archivos CSV se han combinado y exportado a combined_data.xlsx")

KeyboardInterrupt: 

### Filtrando

#### Todos los paises

In [ ]:
countries = ["KOR", "THA", "MYS", "IDN", "ARG", "BRA"]
codes = [7111, 7129, 7211, 7239, 7411, 7429, 7810, 7841]

# Columnas deseadas
columns_to_keep = ["export_value", "import_value", "location_code", "partner_code", "sitc_product_code", "Year"]

# Lista para almacenar DataFrames filtrados
dataframes = []

# Cargar cada archivo CSV, filtrar las filas y agregarlo a la lista de DataFrames
for filename in list_to_use3:
    df = pd.read_csv(filename)

    # Filtrar por códigos en sitc_product_code
    filtered_df = df[df["sitc_product_code"].isin(codes)]

    # Mantener solo las columnas deseadas
    filtered_df = filtered_df[columns_to_keep]

    # Agregar el DataFrame filtrado a la lista
    dataframes.append(filtered_df)

# Combinar todos los DataFrames en uno solo
combined_df = pd.concat(dataframes, ignore_index=True)

In [ ]:
combined_df['location_code'].unique()

In [ ]:
combined_df

,export_value,import_value,location_code,partner_code,sitc_product_code,Year
0,4623.0,36742.0,BRA,AUS,7239,1971
1,0.0,206617.0,IDN,AUS,7239,1971
2,0.0,731.0,KOR,AUS,7239,1971
3,3485.0,424291.0,MYS,AUS,7239,1971
4,563.0,296170.0,THA,AUS,7239,1971
...,...,...,...,...,...,...
222539,1019.0,0.0,ISL,KOR,7129,1967
222540,58373.0,0.0,JPN,KOR,7129,1967
222541,38538.0,0.0,USA,KOR,7129,1967
222542,3993.0,0.0,MYS,PNG,7129,1967


In [ ]:
combined_df[combined_df["sitc_product_code"]==7129]

,export_value,import_value,location_code,partner_code,sitc_product_code,Year
2012,0.0,7075.0,THA,AUT,7129,1971
2013,0.0,96910.0,BRA,BEL,7129,1971
2014,0.0,14482.0,IDN,BEL,7129,1971
2015,0.0,1029.0,THA,BEL,7129,1971
2016,0.0,329244.0,ARG,CHE,7129,1971
...,...,...,...,...,...,...
222539,1019.0,0.0,ISL,KOR,7129,1967
222540,58373.0,0.0,JPN,KOR,7129,1967
222541,38538.0,0.0,USA,KOR,7129,1967
222542,3993.0,0.0,MYS,PNG,7129,1967


In [ ]:
combined_df.to_csv("todos_los_paises.csv", index=False)

export_value da lo mismo que import_value ya que son partidas de comercio internacional. lo que un pais importa el otro lo exporta y visceversa. La solución a este problema esta en el otro notebook.

In [ ]:
combined_df['export_value'].sum()

1568509298228.0

In [ ]:
combined_df['import_value'].sum()

1568509298228.0